In [1]:
import time
import datetime
from scipy.stats import uniform
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge, PoissonRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from IPython.display import display
import xgboost as xgb
from traitlets.traitlets import Any, Dict
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from pandas.core.frame import DataFrame
import pandas as pd
import numpy as np
from enum import Enum, auto
import warnings


warnings.filterwarnings('ignore')
%matplotlib inline

# Pandas settings

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
pd.options.display.width = None
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3



In [3]:
base_path = '../../data/'
encoded_train = pd.read_pickle(f"{base_path}encoded_train.pkl")
encoded_test = pd.read_pickle(f"{base_path}encoded_test.pkl")
encoded_train_dim_red = pd.read_pickle(f"{base_path}encoded_train_dim_red.pkl")
encoded_test_dim_red = pd.read_pickle(f"{base_path}encoded_test_dim_red.pkl")


In [4]:
'''
Convert all the frames to float
'''
encoded_train = encoded_train.apply(lambda x: pd.to_numeric(x))
encoded_test = encoded_test.apply(lambda x: pd.to_numeric(x))


# Modeling

# Modeling Utilities

In [5]:
'''
Models to try [Linear Regression, Generalized Linear Regression, Regularized Regression(Ridge and Lasso Regression), SVM Regression, Tree Based Regression, XgBoost Regression]
    With and without PCA Data
    Hyper Parameter Tuning
    Matrix plotting
    Residual Analysis and Predictions/Error Plotting/Result validation plotting
'''


class DataMode(Enum):
    ENCODED = auto()
    ENCODED_DIM_RED = auto()


def split_data(X, y=None, test_fraction: float = 0.2, shuffle: bool = True, stratify=None):
    return train_test_split(X, y, test_size=test_fraction, random_state=42, shuffle=shuffle, stratify=stratify) if y is not None else train_test_split(X, test_size=test_fraction, random_state=42, shuffle=shuffle, stratify=stratify)


def plot_error_patterns(error_data, **kwargs) -> None:
    fig = ff.create_distplot(hist_data=[error_data["Residual"]], group_labels=["Residual"], **kwargs)
    fig.update_layout(title_text="Residuals Distribution")
    fig.show()

    sm.qqplot(error_data["Residual"], line='45', **kwargs)
    plt
    plt.show()

    fig = px.scatter(error_data, x="Actual", y="Predicted",  trendline="ols", **kwargs)
    fig.update_layout(title_text="Actual vs Predicted values")
    fig.show()

    fig = px.scatter(error_data, x="Predicted", y="Residual",  trendline="ols", **kwargs)
    fig.update_layout(title_text="Predicted values vs Residuals")
    fig.show()

In [6]:


linear = 'linear'
ridge = 'ridge'
lasso = 'lasso'
elasticnet = 'elasticnet'
poisson = 'poisson'
rf = 'rf'
dt = 'dt'
svr = 'svr'
adb = 'adb'
xgbr = 'xgbr'
model_obj = 'model'
params_dict = 'params'
models = {
    linear: {model_obj: LinearRegression(), params_dict: dict(n_jobs=[-1])},
    ridge: {model_obj: Ridge(), params_dict: dict(
        alpha=[0.001, 0.01, 0.1, 1., 10.])},
    lasso: {model_obj: Lasso(),  params_dict: dict(alpha=[0.001, 0.01, 0.1, 1., 10.])},
    elasticnet: {model_obj: ElasticNet(),  params_dict: dict(
        alpha=[0.001, 0.01, 0.1, 0.2, 0.5, 1.],
        l1_ratio=[0.001, 0.01, 0.1, 0.2, 0.5, 1.])},
    # poisson: {model_obj: PoissonRegressor(),  params_dict: dict(
    #     alpha=[0.001, 0.01, 0.1, 0.2, 0.5, 1.],
    #     max_iter=[10, 20, 50, 70, 100],
    #     tol=[1e-4, 1e-3, 1e-2, 1e-1, 1])},
    dt: {model_obj: DecisionTreeRegressor(),  params_dict: dict(
        criterion=["mse", "friedman_mse", "mae", "poisson"],
        splitter=["best", "random"],
        max_depth=[2, 3, 5, 10, 20, 50, 100, 200],
        min_samples_split=[2, 3, 5, 10, 20, 50, 100, 200, 500, 1000],
        max_leaf_nodes=[10, 20, 50, 100, 200, 500, 1000])},
    rf: {model_obj: RandomForestRegressor(),  params_dict: dict(
        n_estimators=[10, 20, 50, 100, 200, 500, 1000],
        criterion=["mse", "mae"],
        max_depth=[2, 3, 5, 10, 20, 50, 100, 200],
        min_samples_split=[2, 3, 5, 10, 20, 50, 100, 200, 500, 1000],
        max_leaf_nodes=[10, 20, 50, 100, 200, 500, 1000])},
    # svr: {model_obj: SVR(),  params_dict: dict(
    #     epsilon=[1e-4, 1e-3, 1e-2, 1e-1, 1e+0, 1e+1],
    #     tol=[1e-4, 1e-3, 1e-2, 1e-1, 1e+0, 1e+1, 1e+2],
    #     C=[1e-4, 1e-3, 1e-2, 1e-1, 1e+0, 1e+1, 1e+2],
    #     loss=['epsilon_insensitive', 'squared_epsilon_insensitive']
    # )},
    adb: {model_obj: AdaBoostRegressor(),  params_dict: dict(
        n_estimators=[10, 20, 50, 100, 200, 500, 1000],
        learning_rate=[1e-4, 1e-3, 1e-2, 1e-1, 1e+0, 1e+1],
        loss=['linear', 'square', 'exponential'],
        random_state=[0]
    )},
    xgbr: {model_obj: xgb.XGBRegressor(), params_dict: dict(
        n_estimators=[10, 20, 50, 100, 200, 500, 1000],
        max_depth=[2, 3, 5, 10, 20, 50, 100, 200],
        learning_rate=[1e-4, 1e-3, 1e-2, 1e-1, 1e+0, 1e+1],
        gamma=[1e-4, 1e-3, 1e-2, 1e-1, 1e+0, 1e+1]
    )}
}


# degrees = [1, 2, 3]
# for degree in degrees:
#     pipeline = Pipeline([('poly_features', PolynomialFeatures(degree=degree)),
#                          ('model', LinearRegression())])
#     pass


# Train test splitResidual Analysis

In [7]:
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = split_data(encoded_train.drop('y', axis=1), encoded_train[['y']])
X_train_dim_red, X_test_dim_red, y_train_dim_red, y_test_dim_red = split_data(encoded_train_dim_red, encoded_train[['y']])

# Class Balancing - NA

# Create model

In [8]:


def prepare_error_data_and_plot(model, X, actual_y: pd.DataFrame, pred_y, mode, model_name, plot: bool = False) -> pd.DataFrame:
    data_mode_str = "without PCA" if mode == DataMode.ENCODED else "with PCA"
    error_df = actual_y.copy()
    error_df["Pred"] = pred_y
    error_df["Res"] = error_df["y"] - error_df["Pred"]
    error_df.columns = ["Actual", "Predicted", "Residual"]
    if plot:
        print(f"Residual Analysis graphs for \"Train Data\" {model_name} {data_mode_str}")
        plot_error_patterns(error_df)
    return error_df

In [9]:
show_graphs = False
all_model_metrics = []
for model, model_config in models.items():
    print(f"Starting for {model.title()}")
    # With and without Dimensionality Reduction
    for mode in [DataMode.ENCODED, DataMode.ENCODED_DIM_RED]:
        X_train, X_test, y_train, y_test = (X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded) if mode == DataMode.ENCODED else (
            X_train_dim_red, X_test_dim_red, y_train_dim_red, y_test_dim_red)
        data_mode_str = "without PCA" if mode == DataMode.ENCODED else "with PCA"
        # Hyper- Parameter tuning
        ra_s_cv = RandomizedSearchCV(model_config.get(model_obj), model_config.get(params_dict), random_state=0, n_jobs=-1, cv=3, verbose=3, return_train_score=True)

        start_time = time.perf_counter()
        ra_s_cv.fit(X_train, y_train)
        end_time = time.perf_counter()

        train_pred = ra_s_cv.predict(X_train)
        test_pred = ra_s_cv.predict(X_test)

        print("-"*50)
        print(f"Best Estimator for {model} {data_mode_str} is {ra_s_cv.best_estimator_}\n")
        print(f"Best Params for {model} {data_mode_str} are {ra_s_cv.best_params_}\n")
        print(f"Cross validation Results for {model} {data_mode_str}\n")
        display(pd.DataFrame(ra_s_cv.cv_results_))
        print("-"*50)

        # Plot evaluation matrix
        prepare_error_data_and_plot(ra_s_cv, X_train, y_train, train_pred, mode, model, show_graphs)
        prepare_error_data_and_plot(ra_s_cv, X_test, y_test, test_pred, mode, model, show_graphs)
        # Record performance
        all_model_metrics.append([f"{model} {data_mode_str}", ra_s_cv.best_score_, ra_s_cv.score(X_train, y_train),
                                  ra_s_cv.score(X_test, y_test.values),  r2_score(y_train, train_pred), r2_score(y_test, test_pred), end_time-start_time])
        print("="*50, "\n")

Starting for Linear
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.0s finished
--------------------------------------------------
Best Estimator for linear without PCA is LinearRegression(n_jobs=-1)

Best Params for linear without PCA are {'n_jobs': -1}

Cross validation Results for linear without PCA



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_jobs,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.125,0.019,0.006,7.663e-04,-1,{'n_jobs': -1},-5.359e+22,-2.441e+22,-9.090e+22,-5.630e+22,2.721e+22,1,0.679,0.619,0.612,0.637,0.03


--------------------------------------------------

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
--------------------------------------------------
Best Estimator for linear with PCA is LinearRegression(n_jobs=-1)

Best Params for linear with PCA are {'n_jobs': -1}

Cross validation Results for linear with PCA

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.5s finished


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_jobs,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.028,0.009,0.002,4.018e-05,-1,{'n_jobs': -1},0.426,0.553,0.562,0.513,0.062,1,0.609,0.532,0.528,0.556,0.038


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
--------------------------------------------------

Starting for Ridge
Fitting 3 folds for each of 5 candidates, totalling 15 fits
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:    0.3s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:    0.9s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.0s finished
--------------------------------------------------
Best Estimator for ridge without PCA is Ridge(alpha=10.0)

Best Params for ridge without PCA are {'alpha': 10.0}

Cross validation Results for ridge without PCA



...

--------------------------------------------------

Starting for Xgbr
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  30 | elapsed:   40.0s remaining:   26.7s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 13.3min finished
--------------------------------------------------
Best Estimator for xgbr without PCA is XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=50,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,37.182,2.040,0.203,0.030,500,3,0.0001,0.01,"{'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.0001, 'gamma': 0.01}",-4.951e+01,-6.005e+01,-6.029e+01,-5.662e+01,5.029e+00,5,-6.018e+01,-5.421e+01,-5.417e+01,-5.619e+01,2.823e+00
1,786.371,10.368,0.044,0.026,1000,200,0.1,0.001,"{'n_estimators': 1000, 'max_depth': 200, 'learning_rate': 0.1, 'gamma': 0.001}",3.401e-01,5.247e-01,2.944e-01,3.864e-01,9.958e-02,2,9.885e-01,9.830e-01,9.909e-01,9.875e-01,3.324e-03
2,2.002,0.500,0.225,0.028,20,20,10,1,"{'n_estimators': 20, 'max_depth': 20, 'learning_rate': 10.0, 'gamma': 1.0}",-7.397e+39,-9.188e+39,-9.296e+39,-8.627e+39,8.711e+38,10,-9.161e+39,-8.252e+39,-8.241e+39,-8.551e+39,4.310e+38
3,539.966,0.418,0.129,0.022,500,50,0.1,0.1,"{'n_estimators': 500, 'max_depth': 50, 'learning_rate': 0.1, 'gamma': 0.1}",3.455e-01,5.225e-01,3.438e-01,4.040e-01,8.385e-02,1,9.884e-01,9.829e-01,9.908e-01,9.874e-01,3.319e-03
4,5.834,0.819,0.193,0.014,50,200,0.01,0.01,"{'n_estimators': 50, 'max_depth': 200, 'learning_rate': 0.01, 'gamma': 0.01}",-1.993e+01,-2.390e+01,-2.399e+01,-2.260e+01,1.893e+00,3,-2.403e+01,-2.166e+01,-2.164e+01,-2.244e+01,1.123e+00
5,1.794,0.149,0.208,0.007,10,50,0.001,0.01,"{'n_estimators': 10, 'max_depth': 50, 'learning_rate': 0.001, 'gamma': 0.01}",-5.363e+01,-6.511e+01,-6.537e+01,-6.137e+01,5.471e+00,8,-6.523e+01,-5.876e+01,-5.872e+01,-6.090e+01,3.061e+00
6,6.075,0.099,0.211,0.005,50,200,0.0001,10,"{'n_estimators': 50, 'max_depth': 200, 'learning_rate': 0.0001, 'gamma': 10.0}",-5.417e+01,-6.577e+01,-6.603e+01,-6.199e+01,5.529e+00,9,-6.589e+01,-5.935e+01,-5.931e+01,-6.152e+01,3.092e+00
7,20.850,0.502,0.200,0.024,200,50,0.0001,1,"{'n_estimators': 200, 'max_depth': 50, 'learning_rate': 0.0001, 'gamma': 1.0}",-5.257e+01,-6.381e+01,-6.406e+01,-6.015e+01,5.357e+00,6,-6.393e+01,-5.759e+01,-5.755e+01,-5.969e+01,3.000e+00
8,17.460,0.291,0.182,0.059,200,10,0.0001,0.1,"{'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.0001, 'gamma': 0.1}",-5.257e+01,-6.381e+01,-6.406e+01,-6.015e+01,5.357e+00,6,-6.393e+01,-5.759e+01,-5.755e+01,-5.969e+01,3.000e+00
9,14.708,1.717,0.160,0.024,200,3,0.001,10,"{'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.001, 'gamma': 10.0}",-3.664e+01,-4.431e+01,-4.448e+01,-4.181e+01,3.655e+00,4,-4.445e+01,-4.004e+01,-4.001e+01,-4.150e+01,2.083e+00


--------------------------------------------------

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  30 | elapsed:   52.2s remaining:   34.8s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  7.1min finished
--------------------------------------------------
Best Estimator for xgbr with PCA is XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=50,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,42.841,7.430,0.079,2.686e-02,500,3,0.0001,0.01,"{'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.0001, 'gamma': 0.01}",-4.952e+01,-6.009e+01,-6.031e+01,-5.664e+01,5.033e+00,5,-6.021e+01,-5.423e+01,-5.419e+01,-5.621e+01,2.826e+00
1,417.781,3.043,0.009,2.533e-03,1000,200,0.1,0.001,"{'n_estimators': 1000, 'max_depth': 200, 'learning_rate': 0.1, 'gamma': 0.001}",3.289e-01,4.981e-01,4.416e-01,4.229e-01,7.033e-02,2,9.885e-01,9.830e-01,9.909e-01,9.875e-01,3.324e-03
2,1.518,0.184,0.036,6.306e-03,20,20,10,1,"{'n_estimators': 20, 'max_depth': 20, 'learning_rate': 10.0, 'gamma': 1.0}",-7.273e+39,-9.167e+39,-9.189e+39,-8.543e+39,8.980e+38,10,-9.007e+39,-8.233e+39,-8.146e+39,-8.462e+39,3.873e+38
3,265.782,15.563,0.038,1.101e-02,500,50,0.1,0.1,"{'n_estimators': 500, 'max_depth': 50, 'learning_rate': 0.1, 'gamma': 0.1}",3.363e-01,4.990e-01,4.392e-01,4.248e-01,6.722e-02,1,9.885e-01,9.830e-01,9.909e-01,9.874e-01,3.322e-03
4,6.417,0.278,0.073,1.404e-02,50,200,0.01,0.01,"{'n_estimators': 50, 'max_depth': 200, 'learning_rate': 0.01, 'gamma': 0.01}",-2.004e+01,-2.410e+01,-2.410e+01,-2.274e+01,1.914e+00,3,-2.416e+01,-2.177e+01,-2.175e+01,-2.256e+01,1.131e+00
5,1.283,0.063,0.041,9.808e-04,10,50,0.001,0.01,"{'n_estimators': 10, 'max_depth': 50, 'learning_rate': 0.001, 'gamma': 0.01}",-5.364e+01,-6.512e+01,-6.537e+01,-6.138e+01,5.472e+00,8,-6.524e+01,-5.876e+01,-5.872e+01,-6.091e+01,3.061e+00
6,6.369,0.486,0.041,8.368e-03,50,200,0.0001,10,"{'n_estimators': 50, 'max_depth': 200, 'learning_rate': 0.0001, 'gamma': 10.0}",-5.418e+01,-6.578e+01,-6.603e+01,-6.199e+01,5.529e+00,9,-6.590e+01,-5.936e+01,-5.932e+01,-6.152e+01,3.092e+00
7,26.757,1.395,0.042,1.412e-02,200,50,0.0001,1,"{'n_estimators': 200, 'max_depth': 50, 'learning_rate': 0.0001, 'gamma': 1.0}",-5.258e+01,-6.383e+01,-6.406e+01,-6.016e+01,5.358e+00,6,-6.394e+01,-5.760e+01,-5.756e+01,-5.970e+01,3.001e+00
8,22.957,2.810,0.044,1.296e-02,200,10,0.0001,0.1,"{'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.0001, 'gamma': 0.1}",-5.258e+01,-6.383e+01,-6.406e+01,-6.016e+01,5.358e+00,6,-6.394e+01,-5.760e+01,-5.756e+01,-5.970e+01,3.001e+00
9,15.105,0.477,0.040,1.298e-02,200,3,0.001,10,"{'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.001, 'gamma': 10.0}",-3.671e+01,-4.445e+01,-4.454e+01,-4.190e+01,3.667e+00,4,-4.454e+01,-4.011e+01,-4.008e+01,-4.158e+01,2.091e+00


--------------------------------------------------



In [10]:
        print(f"Best Params for {model} {data_mode_str} are {ra_s_cv.best_params_}\n")
        print(f"Cross validation Results for {model} {data_mode_str}\n")
        display(pd.DataFrame(ra_s_cv.cv_results_))
        print("-"*50)

        # Plot evaluation matrix
        prepare_error_data_and_plot(ra_s_cv, X_train, y_train, train_pred, mode, model, show_graphs)
        prepare_error_data_and_plot(ra_s_cv, X_test, y_test, test_pred, mode, model, show_graphs)
        # Record performance


Best Params for xgbr with PCA are {'n_estimators': 500, 'max_depth': 50, 'learning_rate': 0.1, 'gamma': 0.1}

Cross validation Results for xgbr with PCA



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,42.841,7.430,0.079,2.686e-02,500,3,0.0001,0.01,"{'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.0001, 'gamma': 0.01}",-4.952e+01,-6.009e+01,-6.031e+01,-5.664e+01,5.033e+00,5,-6.021e+01,-5.423e+01,-5.419e+01,-5.621e+01,2.826e+00
1,417.781,3.043,0.009,2.533e-03,1000,200,0.1,0.001,"{'n_estimators': 1000, 'max_depth': 200, 'learning_rate': 0.1, 'gamma': 0.001}",3.289e-01,4.981e-01,4.416e-01,4.229e-01,7.033e-02,2,9.885e-01,9.830e-01,9.909e-01,9.875e-01,3.324e-03
2,1.518,0.184,0.036,6.306e-03,20,20,10,1,"{'n_estimators': 20, 'max_depth': 20, 'learning_rate': 10.0, 'gamma': 1.0}",-7.273e+39,-9.167e+39,-9.189e+39,-8.543e+39,8.980e+38,10,-9.007e+39,-8.233e+39,-8.146e+39,-8.462e+39,3.873e+38
3,265.782,15.563,0.038,1.101e-02,500,50,0.1,0.1,"{'n_estimators': 500, 'max_depth': 50, 'learning_rate': 0.1, 'gamma': 0.1}",3.363e-01,4.990e-01,4.392e-01,4.248e-01,6.722e-02,1,9.885e-01,9.830e-01,9.909e-01,9.874e-01,3.322e-03
4,6.417,0.278,0.073,1.404e-02,50,200,0.01,0.01,"{'n_estimators': 50, 'max_depth': 200, 'learning_rate': 0.01, 'gamma': 0.01}",-2.004e+01,-2.410e+01,-2.410e+01,-2.274e+01,1.914e+00,3,-2.416e+01,-2.177e+01,-2.175e+01,-2.256e+01,1.131e+00
5,1.283,0.063,0.041,9.808e-04,10,50,0.001,0.01,"{'n_estimators': 10, 'max_depth': 50, 'learning_rate': 0.001, 'gamma': 0.01}",-5.364e+01,-6.512e+01,-6.537e+01,-6.138e+01,5.472e+00,8,-6.524e+01,-5.876e+01,-5.872e+01,-6.091e+01,3.061e+00
6,6.369,0.486,0.041,8.368e-03,50,200,0.0001,10,"{'n_estimators': 50, 'max_depth': 200, 'learning_rate': 0.0001, 'gamma': 10.0}",-5.418e+01,-6.578e+01,-6.603e+01,-6.199e+01,5.529e+00,9,-6.590e+01,-5.936e+01,-5.932e+01,-6.152e+01,3.092e+00
7,26.757,1.395,0.042,1.412e-02,200,50,0.0001,1,"{'n_estimators': 200, 'max_depth': 50, 'learning_rate': 0.0001, 'gamma': 1.0}",-5.258e+01,-6.383e+01,-6.406e+01,-6.016e+01,5.358e+00,6,-6.394e+01,-5.760e+01,-5.756e+01,-5.970e+01,3.001e+00
8,22.957,2.810,0.044,1.296e-02,200,10,0.0001,0.1,"{'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.0001, 'gamma': 0.1}",-5.258e+01,-6.383e+01,-6.406e+01,-6.016e+01,5.358e+00,6,-6.394e+01,-5.760e+01,-5.756e+01,-5.970e+01,3.001e+00
9,15.105,0.477,0.040,1.298e-02,200,3,0.001,10,"{'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.001, 'gamma': 10.0}",-3.671e+01,-4.445e+01,-4.454e+01,-4.190e+01,3.667e+00,4,-4.454e+01,-4.011e+01,-4.008e+01,-4.158e+01,2.091e+00


--------------------------------------------------


,Actual,Predicted,Residual
1073,97.94,95.485,2.455
144,96.41,100.184,-3.774
2380,105.83,104.990,0.840
184,79.09,77.418,1.672
2587,108.69,109.566,-0.876
...,...,...,...
657,113.68,111.256,2.424
3975,88.85,92.392,-3.542
907,89.60,97.462,-7.862
3597,89.23,96.537,-7.307


In [11]:

pd.DataFrame(all_model_metrics, columns=["Algo", "Best Training Score (CV)", "Train Score", "Test Score", "Train R2", "Test R2", "Time Taken"]).style.format({
    "Best Training Score (CV)": "{:.2f}",
    "Train Score": "{:.2f}",
    "Test Score": "{:.2f}",
    "Train R2": "{:.2f}",
    "Test R2": "{:.2f}"
}).hide_index()


Algo,Best Training Score (CV),Train Score,Test Score,Train R2,Test R2,Time Taken
linear without PCA,-56301975154568338604032.00,0.62,-4480720382390470967296.00,0.62,-4480720382390470967296.00,1.254
linear with PCA,0.51,0.55,0.56,0.55,0.56,0.635
ridge without PCA,0.53,0.60,0.58,0.60,0.58,1.140
ridge with PCA,0.52,0.55,0.56,0.55,0.56,0.177
lasso without PCA,0.54,0.55,0.58,0.55,0.58,5.565
lasso with PCA,0.52,0.55,0.56,0.55,0.56,0.182
elasticnet without PCA,0.54,0.60,0.60,0.60,0.60,6.448
elasticnet with PCA,0.52,0.55,0.56,0.55,0.56,0.241
dt without PCA,0.53,0.58,0.58,0.58,0.58,4.223
dt with PCA,0.36,0.51,0.40,0.51,0.40,3.812
